# Integrte p(z)

Probability to be in the background of the cluster i.e. 
$$
P(z > z_l) = \int_{z_l}^{+\infty} dz\ p(z) 
$$
to be higher than a given threshold.

In this notebook, we show the `clmm` functionalities developed to compute:
- those weights under a variety of configuration (true redhsifts or photoz, shape noise or not, ideal galaxy shapes or not).
- the background probability $P(z > z_l)$ for each galaxy

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import scipy

import clmm
from clmm import Cosmology
from clmm import GalaxyCluster
from clmm.dataops import compute_galaxy_weights
from clmm.support import mock_data as mock
clmm.__version__

In [ ]:
cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)

In [ ]:
cluster_z = 0.4

In [ ]:
noisy_data_z = mock.generate_galaxy_catalog(1e14, cluster_z, 4, 
                                                cosmo, 
                                                zsrc = 'chang13', 
                                                delta_so=200, 
                                                massdef='critical',
                                                halo_profile_model='nfw', 
                                                zsrc_min=0.,
                                                zsrc_max=3., 
                                                field_size=10., 
                                                shapenoise=0.5, 
                                                photoz_sigma_unscaled=0.05,
                                                mean_e_err=.1,
                                                ngals=10000)
z_gal = noisy_data_z['ztrue']
z_gal_photoz = noisy_data_z['pzbins']
photoz_pdf_gal = noisy_data_z['pzpdf']
#add redshift dependency on shape measurement error
noisy_data_z['e_err'] = noisy_data_z['e_err']*(1 + 0.4*noisy_data_z['z'])
cl = GalaxyCluster('mock_cluster', 0, 0, .4, noisy_data_z)

## Test p(z) integrals

### From CLMM

In [ ]:
from clmm.dataops import _integ_pzfuncs

In [ ]:
for data in noisy_data_z[:20]:
    plt.plot(data['pzbins'], data['pzpdf'], lw=.5)

In [ ]:
pz_integ = _integ_pzfuncs(noisy_data_z['pzpdf'], noisy_data_z['pzbins'], cluster_z)

### From `qp`

In [ ]:
import qp

In [ ]:
z_grid = np.linspace(0, 5, 100)
qp_dat = qp.Ensemble(
    qp.interp,
    data={'xvals':z_grid,
          'yvals':np.array([np.interp(z_grid, pzbin, pdf)
                for pzbin, pdf in zip(noisy_data_z['pzbins'], noisy_data_z['pzpdf'])])})

In [ ]:
qp_integ = 1-qp_dat.cdf(cluster_z)

### Difference

In [ ]:
plt.scatter(qp_integ, pz_integ, s=2)
plt.plot([0, 1], c='r')
plt.xlabel('cd')